$$
\def\E{\mathbb{E}}
E_1 = \E[\max(0, A(T) - L(1+s))]
$$

$$
B_1 = \E[ \min(L(1+s), A(T)) ] \geq L
$$

$$
E_2 = \E[ \max(0, A(T) + C - L(1+s) - C(1+s')) ] \geq E_1
$$

$$
B_2 = \E[ \min(L(1+s), (A(T) + C) \frac{L}{L + C} ) ]
$$

$$
\begin{align}
B_2' &= \E[ \min(C(1+s'),  (A(T) + C) \frac{C}{L+C}) ] \geq C \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C}, A(T) + C) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C} - C, A(T)) + C ] \\
&= \frac{C}{L+C} \E[ \min((1+s')(L+C) - C, A(T)) ] + \frac{C^2}{L+C} \\
\end{align}
$$

In [20]:
vol = 0.2

In [21]:
a0 = 100

In [139]:
L = 90
C = 15000000

In [140]:
import numpy as np
import scipy as sp
import scipy.stats
import math

In [141]:
def bs(s, k):
    d1 = (math.log(s / k) + (0.5 * vol ** 2)) / vol
    d2 = (math.log(s / k) - (0.5 * vol ** 2)) / vol
    call_price = s * sp.stats.norm.cdf(d1) - k * sp.stats.norm.cdf(d2)   
    put_price = k * sp.stats.norm.cdf(-d2) - s * sp.stats.norm.cdf(-d1)
    return put_price, call_price

In [142]:
def b1(s):
    put, call = bs(a0, L * (1+s))
    return L * (1 + s) - put

In [143]:
b1(0.0)

86.410891883945197

In [144]:
def e1(s):
    put, call = bs(a0, L * (1+s))
    return call

In [145]:
e1(0)

13.589108116054796

In [160]:
res = sp.optimize.minimize(
    lambda s: (b1(s) - L) ** 2,
    [0.0],
    bounds=[(-0.5, 1.5)]
)
res

      fun: array([  5.99730487e-14])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -6.67947732e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 18
      nit: 6
   status: 0
  success: True
        x: array([ 0.0660053])

In [161]:
s = res.x[0]
s

0.066005296922955453

In [162]:
e1(s)

10.00000024489394

In [148]:
def b2prime(sprime):
    strike = (L + C) * (1 + sprime) - C
    put, call = bs(a0, strike)
    return C / (L + C) * (strike - put) + (C ** 2) / (L + C)

In [149]:
b2prime(0.1)

15000009.99994

In [150]:
res = sp.optimize.minimize(
    lambda s: (b2prime(s) - C) ** 2,
    [0.0],
    bounds=[(-0.5, 1.5)]
)
res

      fun: array([ 0.00164565])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2189.62666912])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 58
      nit: 5
   status: 0
  success: True
        x: array([  3.91053100e-07])

In [151]:
sprime = res.x
b2prime(res.x)

array([ 14999999.95943335])

In [152]:
def e2(sprime):
    put, call = bs(a0, L * (1 + s) + C * sprime)
    return call

In [153]:
e2(sprime)

array([ 7.1664379])

In [154]:
e1(s)

array([ 10.00000024])

In [155]:
sprime * C

array([ 5.86579649])